In [ ]:
%matplotlib inline
import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import *
import mido
import numpy
import pandas
import skimage.io

In [ ]:
notes = 24

In [ ]:
input_layer = Input((24, 1))
x = Conv1D(24, 3, padding='same')(input_layer)
x = MaxPool1D(2)(x)
x = Conv1D(12, 3, padding='same')(x)
x = MaxPool1D(2)(x)
x = Conv1D(6, 3, padding='same')(x)
x = MaxPool1D(2)(x)
x = Conv1D(3, 3, padding='same')(x)
x = Activation('sigmoid')(x) # encoded
x = UpSampling1D(2)(x)
x = Conv1D(6, 3, padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(12, 3, padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(1, 3, padding='same')(x)
output_layer = Activation('sigmoid')(x)
model = Model(input_layer, output_layer)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])
model.summary()

In [ ]:
#x = numpy.random.randint(2, size=(32, 36, 1))

In [ ]:
#x

In [ ]:
#model.fit(x, x, epochs=10000)

In [ ]:
#preds = numpy.round(model.predict(x)).astype(int)

In [ ]:
#preds

In [ ]:
#encoded = numpy.round(truncated.predict(x)).astype(int)

In [ ]:
#skimage.io.imshow(encoded[17])

In [ ]:
major = [0, 4, 7, 12]
minor = [0, 3, 7, 12]
diminished = [0, 3, 6, 12]
diminished7 = [0, 3, 6, 9]
augmented = [0, 4, 8, 12]
dominant7 = [0, 4, 7, 10]
minor7 = [0, 3, 7, 10]
major7 = [0, 4, 7, 11]

In [ ]:
def transpose(chords):
    data = []
    for chord in chords:
        for i in range(12):
            data.append(chord + i)
    return numpy.array(data)

In [ ]:
def prepare(chords):
    data = []
    for chord in chords:
        array = numpy.zeros((24, 1))
        for note in chord:
            array[note, 0] = 1
        data.append(array)
    return numpy.array(data)

In [ ]:
def get_meta(names, chords):
    assert len(names) == len(chords)
    meta = []
    for i, chord in enumerate(chords):
        for j in range(12):
            meta.append([names[i], j])
    return meta

In [ ]:
meta = get_meta(['major', 'minor', 'diminished', 'diminished7', 'augmented', 'dominant7', 'minor7', 'major7'], [major, minor, diminished, diminished7, augmented, dominant7, minor7, major7])

In [ ]:
transposed = transpose(numpy.array([major, minor, diminished, diminished7, augmented, dominant7, minor7, major7]))

In [ ]:
prepared = prepare(transposed)

In [ ]:
model.fit(prepared, prepared, epochs=10000)

In [ ]:
input2 = Input((24, 1))
x2 = Conv1D(24, 3, padding='same', weights=model.layers[1].get_weights())(input2)
x2 = MaxPool1D(2)(x2)
x2 = Conv1D(12, 3, padding='same', weights=model.layers[3].get_weights())(x2)
x2 = MaxPool1D(2)(x2)
x2 = Conv1D(6, 3, padding='same', weights=model.layers[5].get_weights())(x2)
x2 = MaxPool1D(2)(x2)
x2 = Conv1D(3, 3, padding='same', weights=model.layers[7].get_weights())(x2)
output2 = Activation('sigmoid')(x2)
truncated = Model(input2, output2)
truncated.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])
truncated.summary()

In [ ]:
features = truncated.predict(prepared)
flattened = numpy.empty((len(features), 9))
for i, feature in enumerate(features):
    flattened[i, :] = feature.flatten()

In [ ]:
skimage.io.imshow(features[0, numpy.newaxis])

In [ ]:
datadf = pandas.DataFrame(flattened)

In [ ]:
datadf

In [ ]:
metadf = pandas.DataFrame(data=meta, columns=['Chord', 'Transposition'])

In [ ]:
metadf

In [ ]:
df = pandas.concat([metadf, datadf], axis=1)

In [ ]:
df

In [ ]:
#df.to_csv('/home/santiago/Projects/MusicGenerator/csv/chords.csv', index=False)

In [ ]:
df.to_csv('/home/santiago/Projects/MusicGenerator/csv/chords.txt', sep='\t', index=False)